In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909,0.100000,2.110,0.100000
1,1,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,2,5.7,joana wälti,2,Joana Wälti,2.0,2.102,0.124,6.882701,1.809328,2.053610,0.122141,0.122141,2.102,0.122141
2,2,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,3,7.3,anna kindlundh,3,Anna Kindlundh,6.0,2.077,0.120,6.107582,1.726928,2.056589,0.164830,0.164830,2.077,0.120000
3,3,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,4,7.9,jeanette jönsson hellstadius,4,Jeanette Jönsson Hellstadius,3.0,2.041,0.134,7.474857,1.845341,2.089668,0.122164,0.122164,2.041,0.134000
4,4,30,Suunta Jyväskylä 1,Suunta Jyväskylä,FIN,1,6.0,veera klemettinen,1,Veera Klemettinen,6.0,2.064,0.106,7.198831,1.873431,2.050207,0.088388,0.100000,2.064,0.106000


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,8,Södertälje Nykvarn Orientering 1,1,6.0,Fiona Bunn,2.110,0.100000
1,8,Södertälje Nykvarn Orientering 1,2,5.7,Joana Wälti,2.102,0.122141
2,8,Södertälje Nykvarn Orientering 1,3,7.3,Anna Kindlundh,2.077,0.120000
3,8,Södertälje Nykvarn Orientering 1,4,7.9,Jeanette Jönsson Hellstadius,2.041,0.134000
4,30,Suunta Jyväskylä 1,1,6.0,Veera Klemettinen,2.064,0.106000


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                    \
leg                                     1                                 2   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
37                    Farum-Tisvilde OK 1               Farum-Tisvilde OK 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
52                            Byåsen IL 1                       Byåsen IL 1   

                                                                             \
leg                                     3                                 4   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
37                    Farum-Tisvilde OK 1               Farum-Tisvilde OK 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
52                            Byåsen IL 1                       Byåsen IL 1   

        dist                              name                    \
leg        1    2    3    4                  1                 2   
team_id                                                            
8        6.0  5.7  7.3  7.9         Fiona Bunn       Joana Wälti   
30       6.0  5.7  7.3  7.9  Veera Klemettinen  Elisa Viholainen   
37       6.0  5.7  7.3  7.9       Agnes Kracht       Iben Valery   
41       6.0  5.7  7.3  7.9     Laura Ramstein  Lotta Kirvesmies   
52       6.0  5.7  7.3  7.9      Ingeborg Eide      Emma Arnesen   

                                                          log_mean            \
leg                       3                             4        1         2   
team_id                                                                        
8            Anna Kindlundh  Jeanette Jönsson Hellstadius    2.110  2.102000   
30       Barbora Chaloupska             Kaisa Klemettinen    2.064  2.061000   
37          Mathilde Kracht                   Masha Semak    2.131  1.983129   
41        Anna-Kaisa Anjala                  Minna Kauppi    1.973  1.968000   
52               Göril Eide                 Jenny Persson    2.069  2.023000   

                        log_std                             
leg          3      4         1         2         3      4  
team_id                                                     
8        2.077  2.041  0.100000  0.122141  0.120000  0.134  
30       1.984  2.050  0.106000  0.122000  0.140614  0.138  
37       2.073  2.108  0.106113  0.123215  0.156757  0.197  
41       2.024  2.025  0.116000  0.097000  0.147421  0.109  
52       2.128  2.295  0.095000  0.102000  0.121845  0.206

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(633, 20)

(622, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict, **fin_sum_log_means_dict, **fin_sum_log_stds_dict, **mass_start_dict, **dark_during_leg_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,fin_sum_log_std_3,fin_sum_log_std_4,mass_start_1,mass_start_2,mass_start_3,mass_start_4,dark_during_leg_1,dark_during_leg_2,dark_during_leg_3,dark_during_leg_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,40.718579,82.659504,135.737844,191.505197,60.174076,112.440400,177.005417,244.635085,49.405167,96.411294,...,0.067110,0.061572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,38.329052,78.749252,126.596807,182.654233,58.347053,108.796620,168.403922,236.056828,47.272866,92.376284,...,0.072339,0.064936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,41.175710,78.911305,129.581532,185.282487,62.162800,108.176679,176.555554,255.396852,50.657646,92.354719,...,0.078329,0.081122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,34.196188,72.293580,121.358334,177.835882,54.051432,97.773764,162.721901,226.564385,43.109795,84.025135,...,0.074099,0.061437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,39.368723,79.384797,134.811527,198.669507,57.282515,104.184284,173.476089,275.933500,47.553563,90.958590,...,0.063890,0.082633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()

,team_id,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,...,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,...,Anna Kindlundh,Jeanette Jönsson Hellstadius,2.110000,2.102000,2.077000,2.041000,0.100000,0.122141,0.120000,0.134000
1,30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,...,Barbora Chaloupska,Kaisa Klemettinen,2.064000,2.061000,1.984000,2.050000,0.106000,0.122000,0.140614,0.138000
2,37,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,6.0,5.7,7.3,7.9,Agnes Kracht,...,Mathilde Kracht,Masha Semak,2.131000,1.983129,2.073000,2.108000,0.106113,0.123215,0.156757,0.197000
3,41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,...,Anna-Kaisa Anjala,Minna Kauppi,1.973000,1.968000,2.024000,2.025000,0.116000,0.097000,0.147421,0.109000
4,52,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,6.0,5.7,7.3,7.9,Ingeborg Eide,...,Göril Eide,Jenny Persson,2.069000,2.023000,2.128000,2.295000,0.095000,0.102000,0.121845,0.206000
5,58,Northern Wind 1,Northern Wind 1,Northern Wind 1,Northern Wind 1,6.0,5.7,7.3,7.9,Anastassia Grigorieva,...,Natalia Kondrashova,Svetlana Raikova,2.235949,2.036000,2.215000,1.938000,0.100000,0.136361,0.142952,0.099000
6,68,Asker SK 1,Asker SK 1,Asker SK 1,Asker SK 1,6.0,5.7,7.3,7.9,Synne Strand,...,Anna Öfsthus Gravir,Marit Isachsen,2.139000,2.121000,2.274000,2.307000,0.130934,0.236187,0.182309,0.235000
7,78,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,6.0,5.7,7.3,7.9,Tytti Kirvesmies,...,Anniina Taipale,Saara Kukkonen,1.986000,2.126671,2.276000,1.997000,0.105000,0.164399,0.164521,0.125000
8,80,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,6.0,5.7,7.3,7.9,Mari Eidsmo,...,Dina Reitan,Vera Melkild,2.081351,2.068919,2.090712,2.102000,0.167446,0.142361,0.159152,0.131051
9,83,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,6.0,5.7,7.3,7.9,Laura King,...,Sofia Wigle,Anna Ek Larsson,2.070204,2.055800,2.060000,2.088466,0.137841,0.140509,0.160034,0.134402


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,fin_sum_log_std_3,fin_sum_log_std_4,mass_start_1,mass_start_2,mass_start_3,mass_start_4,dark_during_leg_1,dark_during_leg_2,dark_during_leg_3,dark_during_leg_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,...,0.067110,0.061572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,...,0.072339,0.064936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,6.0,5.7,7.3,7.9,Agnes Kracht,Iben Valery,...,0.078329,0.081122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,Lotta Kirvesmies,...,0.074099,0.061437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,6.0,5.7,7.3,7.9,Ingeborg Eide,Emma Arnesen,...,0.063890,0.082633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   
        2                  Suunta Jyväskylä 1   5.7   
        3                  Suunta Jyväskylä 1   7.3   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.718579   
        2                     Joana Wälti     2.102  0.122141    82.659504   
        3                  Anna Kindlundh     2.077  0.120000   135.737844   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.505197   
30      1               Veera Klemettinen     2.064  0.106000    38.329052   
        2                Elisa Viholainen     2.061  0.122000    78.749252   
        3              Barbora Chaloupska     1.984  0.140614   126.596807   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.174076   49.405167   49.682459          3.900754   
        2    112.440400   96.411294   96.610173          4.567556   
        3    177.005417  154.868800  155.264296          5.042877   
        4    244.635085  216.145254  216.650612          5.376389   
30      1     58.347053   47.272866   47.551354          3.856060   
        2    108.796620   92.376284   92.673078          4.525726   
        3    168.403922  145.774716  146.186451          4.982264   

             fin_sum_log_std  mass_start  dark_during_leg  
team_id leg                                                
8       1           0.098952         0.0              0.0  
        2           0.079071         0.0              0.0  
        3           0.067110         0.0              0.0  
        4           0.061572         0.0              0.0  
30      1           0.107166         0.0              0.0  
        2           0.081828         0.0              0.0  
        3           0.072339         0.0              0.0

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates.head()

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.718579   
        2                     Joana Wälti     2.102  0.122141    82.659504   
        3                  Anna Kindlundh     2.077  0.120000   135.737844   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.505197   
30      1               Veera Klemettinen     2.064  0.106000    38.329052   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.174076   49.405167   49.682459          3.900754   
        2    112.440400   96.411294   96.610173          4.567556   
        3    177.005417  154.868800  155.264296          5.042877   
        4    244.635085  216.145254  216.650612          5.376389   
30      1     58.347053   47.272866   47.551354          3.856060   

             fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                 
8       1           0.098952         0.0              0.0   
        2           0.079071         0.0              0.0   
        3           0.067110         0.0              0.0   
        4           0.061572         0.0              0.0   
30      1           0.107166         0.0              0.0   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
8       1   2019-06-15 14:49:24.310019732 2019-06-15 14:40:43.114742994   
        2   2019-06-15 15:36:24.677651167 2019-06-15 15:22:39.570228815   
        3   2019-06-15 16:34:52.128016471 2019-06-15 16:15:44.270664930   
        4   2019-06-15 17:36:08.715228796 2019-06-15 17:11:30.311846972   
30      1   2019-06-15 14:47:16.371978283 2019-06-15 14:38:19.743098021   

                           fin_time_end95  
team_id leg                                
8       1   2019-06-15 15:00:10.444572687  
        2   2019-06-15 15:52:26.423979282  
        3   2019-06-15 16:57:00.324992418  
        4   2019-06-15 18:04:38.105082273  
30      1   2019-06-15 14:58:20.823151112

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["fin_time_median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["fin_time_start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["fin_time_end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name', 
 'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_print[for_print['team'].str.contains("Reak")]

,team,leg,name,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
team_id,,,,,,,,


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
312,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.988450,0.116728,2019-06-15 11:42:50.597743750+00:00,2019-06-15 12:07:47.723588228+00:00,2019-06-15 11:53:59.347073078+00:00,0.0000,0.0
313,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.744576,0.092515,2019-06-15 12:36:13.833943367+00:00,2019-06-15 13:18:19.704730034+00:00,2019-06-15 12:54:57.003420591+00:00,0.0000,0.0
314,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.277376,0.083138,2019-06-15 13:46:19.418945074+00:00,2019-06-15 14:51:07.773175716+00:00,2019-06-15 14:15:31.635554552+00:00,0.0000,0.0
315,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.700775,0.082658,2019-06-15 15:13:58.053776264+00:00,2019-06-15 16:52:47.449332476+00:00,2019-06-15 15:59:04.750211+00:00,0.0002,0.0
560,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.208776,0.163736,2019-06-15 11:48:34.512257814+00:00,2019-06-15 12:32:25.415865183+00:00,2019-06-15 12:07:13.018993616+00:00,0.0000,0.0
561,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.794824,0.110975,2019-06-15 12:37:24.431091547+00:00,2019-06-15 13:30:12.133892298+00:00,2019-06-15 13:00:51.984657288+00:00,0.0000,0.0
562,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.335518,0.102265,2019-06-15 13:49:27.890407562+00:00,2019-06-15 15:14:14.764888048+00:00,2019-06-15 14:27:25.044646502+00:00,0.0000,0.0
563,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.680042,0.080382,2019-06-15 15:11:02.173353195+00:00,2019-06-15 16:43:19.095087767+00:00,2019-06-15 15:52:51.560749292+00:00,0.0057,0.0
828,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.372330,0.142811,2019-06-15 11:59:55.987347126+00:00,2019-06-15 12:45:04.240144491+00:00,2019-06-15 12:19:14.556025028+00:00,0.0000,0.0
829,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.065462,0.111728,2019-06-15 13:07:41.242549896+00:00,2019-06-15 14:17:28.584559202+00:00,2019-06-15 13:38:24.630582333+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")